<a href="https://colab.research.google.com/github/mphirke/fire-emblem-fake-portaits-GBA/blob/master/Fire_Emblem_StyleGAN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN2: training a Fire Emblem GBA portait model
References:
- Steam StyleGAN2 by woctezuma: https://github.com/woctezuma/steam-stylegan2
- the original StyleGAN2 repository: https://github.com/NVlabs/stylegan2
- woctezuma's fork of StyleGAN2: https://github.com/woctezuma/stylegan2 for easy saving results in google drive.



### Request more RAM (once per session)


It might not be required but if your runtime crashes, 
uncomment and run the following cell if you do not have Colab high RAM (26 GB).

To have access to more memory, crash the Google Colab session  once.

References:
-   https://github.com/googlecolab/colabtools/issues/253#issuecomment-551056637
-   https://colab.research.google.com/drive/1dBN-wwYUngLYVt985wGs_OKPlK_ANB9D

Remember to comment it out after you get high RAM session (if you need it) else your session will keep crashing.


In [ ]:
# d = [ '0' ]
# while(True):
#   d += d

### Check the GPU

In [ ]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-e7aa78eb-f344-2456-a236-b8e368afc717)


### Switch to Tensorflow 1.x

The default TensorFlow version in Colab switched from  1.x to 2.x on the 27th of March, 2020.

Reference: https://colab.research.google.com/notebooks/tensorflow_version.ipynb

We switch to version 1.x to avoid the following error:
> `ModuleNotFoundError: No module named 'tensorflow.contrib' colab`

In [ ]:
%tensorflow_version 1.x

The following code downloads my preprocessed data. If you have your own custom images, I have attachted python scripts in Github repo (Read Github README) to know how to preprocess your own custom images to feed to the model.

In [ ]:
!wget https://www.dropbox.com/s/kxs71laczusdx4g/FE_vanilla.zip
!unzip -o FE_vanilla.zip

--2020-07-30 11:18:35--  https://www.dropbox.com/s/kxs71laczusdx4g/FE_vanilla.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/kxs71laczusdx4g/FE_vanilla.zip [following]
--2020-07-30 11:18:35--  https://www.dropbox.com/s/raw/kxs71laczusdx4g/FE_vanilla.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7f727cb6536d68a5b8e4c21b90.dl.dropboxusercontent.com/cd/0/inline/A8iB1LDFtpATQCj3ZP_SwvalAOB1SEw7jx0t1UBOS9V3kKo4pbZSmj4Bs9NDJN-7ZgpY8MzPruGZZbZYYP1t31Ds5MqQQkg1jYpWenDwVE48ypMKQ_yYu3mzT3IOduTo6Xo/file# [following]
--2020-07-30 11:18:35--  https://uc7f727cb6536d68a5b8e4c21b90.dl.dropboxusercontent.com/cd/0/inline/A8iB1LDFtpATQCj3ZP_SwvalAOB1SEw7jx0t1UBOS9V3kKo4pbZSmj4Bs9NDJN-7ZgpY8MzPruGZZbZYYP1t31Ds5MqQQkg1jYpWenDwVE48

## Installing StyleGAN2

### woctezuma's fork of the official StyleGAN2 implementation

In [ ]:
%cd /content/

/content


In [ ]:
!rm -rf stylegan2/
!git clone https://github.com/woctezuma/stylegan2.git

Cloning into 'stylegan2'...
remote: Enumerating objects: 120, done.
remote: Total 120 (delta 0), reused 0 (delta 0), pack-reused 120
Receiving objects: 100% (120/120), 586.40 KiB | 2.41 MiB/s, done.
Resolving deltas: 100% (51/51), done.


In [ ]:
pushd

['/content']

In [ ]:
%cd stylegan2/

/content/stylegan2


Switch to woctezuma fork of StyleGAN2, designed for Google Colab:

1.   google-colab
2.   google-colab-save-every-tick


-   `google-colab` to save every 2 ticks,
-   `google-colab-save-every-tick` to save every tick.



NB: It is expected that 1 tick ~ 15 min. However, if you are unlucky with the Colab machine which is allotted, 1 tick ~ 1 hour.

You can look at the progress of your trining in your Google Drive in folder results. Both the generated fakes as well as the training .pkl file will be stored there, HOWEVER, each tick will store an approx 300mb of data in your Google Drive so if you are limited on space, I would advise use `google-colab` instead of `google-colab-save-every-tick`

In [ ]:
!git checkout google-colab-save-every-tick

Branch 'google-colab-save-every-tick' set up to track remote branch 'google-colab-save-every-tick' from 'origin'.
Switched to a new branch 'google-colab-save-every-tick'


In [ ]:
!nvcc test_nvcc.cu -o test_nvcc -run

CPU says hello.
GPU says hello.


In [ ]:
popd

/content
popd -> /content


## Mounting Google Drive

In [ ]:
!pip install Google-Colab-Transfer

In [ ]:
import colab_transfer

colab_path = colab_transfer.get_path_to_home_of_local_machine()
drive_path = colab_transfer.get_path_to_home_of_google_drive()

print('Disk of the virtual matchine: {}'.format(colab_path))
print('Google Drive: {}'.format(drive_path))

Disk of the virtual matchine: /content/
Google Drive: /content/drive/My Drive/


In [ ]:
colab_transfer.mount_google_drive()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Data parameters

In [ ]:
folder_name = '/content/FE_vanilla'

## Preparing datasets (once)

### Preparing data for StyleGAN2

In [ ]:
!python stylegan2/dataset_tool.py create_from_images '/content/datasets/fire_emblem' $folder_name

Loading images from "/content/FE_vanilla"
Creating dataset "/content/datasets/fire_emblem"
Added 499 images.


## Training networks

There is no need to edit `training/training_loop.py`, thanks to automatic resuming from the latest snapshot, implemented in my fork.

Otherwise, one would have to manually edit the file from within the Google Colab session!

### Train with the official StyleGAN2 implementation

- `--mirror-augment=true`: data augmentation with horitontal mirroring,
- `--total-kimg=350`: **350k images processed (I am keeping it 350k in this notebook because otherwise Colab runs out of time, but ideally you want this value to be higher. My results at 350k img were mediocre. Because of woctezuma's stylegan2 colab implementation, the resulted model gets stored in your google drive after every tick, so even if you run out of time and restart the process, the model will start from where it left.**
 You most likely want to keep it 500+ and then run this notebook as many as times it takes to complete those number of kimgs. Don't keep the value too high or the model will overfit (I started overfitting at about 700k img).**


Model snapshots are directly saved to Google Drive (`--result-dir='/content/drive/My Drive/results_FE'`). 

### You can look at the progress of your trining in your Google Drive in folder results. Both the generated fakes as well as the training .pkl file are stored there, HOWEVER, each tick will store an approx 300mb of data in your Google Drive if you are using the fork 

In [ ]:
!python stylegan2/run_training.py --config=config-e --metrics=none \
   --data-dir='/content/datasets' --dataset=fire_emblem \
   --mirror-augment=true \
   --total-kimg=700 \
   --result-dir='/content/drive/My Drive/results_FE' \


Local submit - run_dir: /content/drive/My Drive/results_FE/00001-stylegan2-fire_emblem-1gpu-config-e
dnnlib: Running training.training_loop.training_loop() on localhost...
Streaming data using training.dataset.TFRecordDataset...
tcmalloc: large alloc 4294967296 bytes == 0x7b5c000 @  0x7fda8c5e0001 0x7fda8906c765 0x7fda890d0bb0 0x7fda890d2a4f 0x7fda89169048 0x50a635 0x50cd96 0x507d64 0x509042 0x594931 0x549e5f 0x5513d1 0x59fc4e 0x50d356 0x507d64 0x588e5c 0x59fc4e 0x50d356 0x507d64 0x588e5c 0x59fc4e 0x50d356 0x509758 0x50a48d 0x50bfb4 0x509758 0x50a48d 0x50bfb4 0x507d64 0x588e5c 0x59fc4e
tcmalloc: large alloc 4294967296 bytes == 0x7fd8c3ac6000 @  0x7fda8c5de1e7 0x7fda8906c5e1 0x7fda890d0c78 0x7fda890d0f37 0x7fda89168f28 0x50a635 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x509758 0x50a48d 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64
tcmalloc: large alloc 4294967296 bytes

## Generate Images

Generate 1000 images (Seeds 0 to 999). Use unique/different seeds if you want different results.

Psi is the basically the amount of truncation we do on the results. If psi is 1, there is no untruncation (that is, even the bad results are generated). MORE variety but WORSE results.

For example, If psi is 0.5, it means that we are truncating the results that are very far off from our original dataset so with truncation_psi set to 0.5, we would have LESS variety but BETTER results.

### Replace the pickel path variable with path to your the .pkl file of the model that you want to use.

### PAY ATTENTION TO THE \ AFTER MY IN THE PATH. Make sure you put it after My after replacing the link otherwise the space will be treated as a different argument. 

In [ ]:
# pickle_path = '''/content/drive/My\ Drive/results/00003-stylegan2-steam-1gpu-config-e/network-snapshot-000697.pkl'''
# !python stylegan2/run_generator.py generate-images --network=$pickle_path \
#   --seeds=0-999 --truncation-psi=1.0

# The results are stored in the results/ 